# Load DNS and mDNS datasets

In [1]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch_geometric.transforms as T

from libs.loader import DNS
from libs.utils import score

In [2]:
cuda_device = 3

if torch.cuda.is_available():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(cuda_device)
    
torch.manual_seed(42)

## Load Graphs

In [3]:
kg_path = lambda graph_name: f'./data/{graph_name}'

#### mDNS

In [4]:
# dataset = DNS(root=kg_path('mDNS'), transform=T.Compose([T.NormalizeFeatures(), T.ToUndirected()]), balance_gt=True)
# data = dataset[0]
# data

#### DNS

In [5]:
dataset = DNS(root=kg_path('DNS'), transform=T.Compose([T.NormalizeFeatures(), T.ToUndirected()]), balance_gt=True)
data = dataset[0]
print(data['domain_node'].y.unique(return_counts=True))
print(data['domain_node'].train_mask.unique(return_counts=True))

Remove parallel edges: type
similar    50910
dtype: int64
(tensor([0, 1, 2]), tensor([  4386,  19953, 349136]))
(tensor([False,  True]), tensor([369089,   4386]))


In [6]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv, SAGEConv, HeteroConv, Linear


class HeteroGNN(torch.nn.Module):
    def __init__(self, metadata, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                edge_type: SAGEConv((-1, -1), hidden_channels)
                for edge_type in metadata[1]
            })
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: F.leaky_relu(x) for key, x in x_dict.items()}
        return self.lin(x_dict['domain_node'])
    

model = HeteroGNN(data.metadata(), hidden_channels=64, out_channels=2,
                  num_layers=2)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

with torch.no_grad():  # Initialize lazy modules.
    out = model(data.x_dict, data.edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['domain_node'].train_mask
    loss = F.cross_entropy(out[mask], data['domain_node'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    with torch.no_grad():
        pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask', 'test_mask']:
        mask = data['domain_node'][split]
        acc = (pred[mask] == data['domain_node'].y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs


for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
              f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')
    
model.eval()
with torch.no_grad():
    pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)
mask = data['domain_node']['test_mask']
scores = score(pred[mask],data['domain_node'].y[mask])
for metric, score in scores.items():
    print(metric, ':{:.2f}'.format(score))

Epoch: 020, Loss: 0.2897, Train: 0.8940, Val: 0.8963, Test: 0.8978
Epoch: 040, Loss: 0.2398, Train: 0.9111, Val: 0.9066, Test: 0.9103
Epoch: 060, Loss: 0.2134, Train: 0.9220, Val: 0.9168, Test: 0.9126
Epoch: 080, Loss: 0.1965, Train: 0.9220, Val: 0.9060, Test: 0.9099
Epoch: 100, Loss: 0.1772, Train: 0.9382, Val: 0.9179, Test: 0.9206
Epoch: 120, Loss: 0.1675, Train: 0.9398, Val: 0.9231, Test: 0.9255
Epoch: 140, Loss: 0.1642, Train: 0.9403, Val: 0.9248, Test: 0.9270
Epoch: 160, Loss: 0.1543, Train: 0.9441, Val: 0.9242, Test: 0.9263
Epoch: 180, Loss: 0.1528, Train: 0.9435, Val: 0.9208, Test: 0.9255
Epoch: 200, Loss: 0.1446, Train: 0.9478, Val: 0.9259, Test: 0.9297
tn, fp, fn, tp 1204 84 101 1242
tn, fp, fn, tp 1204 84 101 1242
acc :0.93
f1 :0.93
auc :0.93
prec :0.94
recall :0.92
fpr :0.07
